In [1]:
import os.path as op
import time

import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

# Data Parsing

In [2]:
def parse_data(fpath):
    data = np.genfromtxt(fpath, dtype=np.int64, skip_header=7)
    X, y = data[:, :-1], data[:, -1]
    return X, y

In [3]:
dirname = op.abspath("")
fpath = op.join(dirname, "stable6.txt")

X, y = parse_data(fpath)
y -= 1  # Index classes starting at 0 for CrossEntropyLoss() to work

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Neural Network

In [4]:
class My_Net(nn.Module): 
    def __init__(self, n_features, n_classes, hidden_dim1=15, hidden_dim2=15):
        super(My_Net, self).__init__()
        #feed forward layers
        self.layer_1 = nn.Linear(n_features, hidden_dim1)
        self.layer_2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer_3 = nn.Linear(hidden_dim2, n_classes)

        #activations
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.softmax(out)
        return out

## Initialization

In [5]:
# Fix random seed
torch.manual_seed(0)

n_classes = len(np.unique(y))
n_features = X_train.shape[1]

# Hyperparameters to tune
hidden_dim1 = 40
hidden_dim2 = 30
lr = 5e-3

if "stable4" in fpath:
    batch_size = 1
elif "stable5" in fpath:
    batch_size = 32
elif "stable6" in fpath:
    batch_size = 64

net = My_Net(n_features, n_classes, hidden_dim1, hidden_dim2)

# Standard cross entropy loss for multi-classification tasks
loss = nn.CrossEntropyLoss()

# Define the optimizer. Here we use Adam optimizer.
opt = torch.optim.Adam(net.parameters(), lr=lr)

In [6]:
Xtrain = torch.Tensor(X_train)
Xtest = torch.Tensor(X_test)
ytrain = torch.LongTensor(y_train)
ytest = torch.LongTensor(y_test)

In [7]:
train = torch.utils.data.TensorDataset(Xtrain, ytrain)
test = torch.utils.data.TensorDataset(Xtest, ytest)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)

In [8]:
# Initialize the network using Xavier initialization.
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

In [9]:
net.apply(init_weights)

My_Net(
  (layer_1): Linear(in_features=15, out_features=40, bias=True)
  (layer_2): Linear(in_features=40, out_features=30, bias=True)
  (layer_3): Linear(in_features=30, out_features=6, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

## Training

In [10]:
def train_eval(net, train_loader, verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0

    for inputs, labels in train_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs, labels).item()
        num_batches += 1

    avg_loss = loss_sum / num_batches
    accuracy = correct.item() / total
        
    if verbose:
        print(f"Train accuracy: {accuracy * 100:.4f} %")

    return avg_loss, accuracy

def test_eval(net, test_loader, verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0

    for inputs, labels in test_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs, labels).item()
        num_batches += 1

    avg_loss = loss_sum / num_batches
    accuracy = correct.item() / total
        
    if verbose:
        print(f"Test accuracy: {accuracy * 100:.4f} %")

    return avg_loss, accuracy

In [11]:
epochs = 25
train_loss_store = []
train_acc_store = []
test_loss_store = []
test_acc_store = []

for epoch in range(epochs):
    time1 = time.time()
    print(f"In epoch {epoch+1} : ")
    for i, (sample, label) in enumerate(train_loader):
        # Set the gradients to zero initially for each batch
        opt.zero_grad()
        outputs = net(sample)
        l = loss(outputs, label)
        l.backward()
        opt.step()
    
    l_temp, acc_temp = train_eval(net, train_loader)
    train_loss_store.append(l_temp)
    train_acc_store.append(acc_temp)

    l_temp, acc_temp = test_eval(net, test_loader)
    test_loss_store.append(l_temp)
    test_acc_store.append(acc_temp)

    time2 = time.time()
    print(f"Time lapse: {round((time2-time1), 2):.3f} secs")

In epoch 1 : 
Train accuracy: 66.6173 %
Test accuracy: 66.6972 %
Time lapse: 1.530 secs
In epoch 2 : 
Train accuracy: 66.6173 %
Test accuracy: 66.6972 %
Time lapse: 1.440 secs
In epoch 3 : 
Train accuracy: 69.5688 %
Test accuracy: 69.8708 %
Time lapse: 1.370 secs
In epoch 4 : 
Train accuracy: 73.5536 %
Test accuracy: 73.2072 %
Time lapse: 1.330 secs
In epoch 5 : 
Train accuracy: 74.9967 %
Test accuracy: 74.6414 %
Time lapse: 1.300 secs
In epoch 6 : 
Train accuracy: 76.1259 %
Test accuracy: 75.6688 %
Time lapse: 1.310 secs
In epoch 7 : 
Train accuracy: 76.6578 %
Test accuracy: 76.2181 %
Time lapse: 1.330 secs
In epoch 8 : 
Train accuracy: 76.9761 %
Test accuracy: 76.5334 %
Time lapse: 1.280 secs
In epoch 9 : 
Train accuracy: 78.0311 %
Test accuracy: 77.5099 %
Time lapse: 1.310 secs
In epoch 10 : 
Train accuracy: 78.6371 %
Test accuracy: 77.9473 %
Time lapse: 1.290 secs
In epoch 11 : 
Train accuracy: 79.2867 %
Test accuracy: 78.6695 %
Time lapse: 1.290 secs
In epoch 12 : 
Train accuracy: